In [1]:
import pandas as pd
import zipfile
import xlrd
import re
import os
import matplotlib.pyplot as plt
from functools import reduce

In [2]:
x=zipfile.ZipFile(f'..//Final_Project/Altadis/FY1920/Master_Cigarrillos_FY1920(Septiembre).zip')
df=pd.read_excel(x.open('Master_Cigarrillos_FY1920(Septiembre).xlsx'))
keep=[i for i in df.columns[0:33]]
delete=['Alias_Tienda','Promotor_Responsable','ASM','TOTAL JPS', 'TOTAL HABANOS', 'TOTAL BN','TOTAL GAULOISES','B_N_ BEIGE 20S','TOTAL GAULOISES','TOTAL JPS','CLASSIFICATION','Ipos',"B_N_ CLASICO 20S",'B_N_ CLASICO BLANDO 20S','AREA','area','Area','sapcode','SAPCODE','TOTAL BN','TOTAL HABANOS','logista_segment','BASTOS FILTRE 25S','BONCALO 20S','BROOKLYN AZUL KS 20S','BROOKLYN ROJO KS 20S']
rename={'NAME':'Name','name':'Name','CUSTOMERCODE':'customer_id','sbl_frequency':'Visit Frequency','PROVINCE':'province','PostalCode':'Zip code','Siebel_segment':'Domestic/Border','SBL_FREQUENCY':'Visit Frequency','LOGISTA_SEGMENT':'logista_segment','POSTALCODE':'PostalCode','ESTADO':'Estado','estado':'Estado','CITY':'City','city':'City'}

In [3]:
x=zipfile.ZipFile(f'..//Final_Project/Altadis/FY1920/Master_Picadura_FY1920(Septiembre).zip')
dfp=pd.read_excel(x.open('Master_Picadura_FY1920(Septiembre).xlsx'))
keepp=[i for i in dfp.columns[0:24]]

In [4]:
lista=[]
data_cigarrillos=pd.DataFrame()
def concat_files(x):
    for dirpath, dirnames, filenames in os.walk(x):
        for filename in filenames:
            for i in range(10,20):
                if filename.endswith(f"Master_Cigarrillos_FY{i}{i+1}(Septiembre).zip"):
                    print(f'FY{i}{i+1}')
                    path=os.path.join(dirpath, filename)
                    zipp=zipfile.ZipFile(path)
                    for y in zipp.namelist():
                        data=pd.read_excel(zipp.open(y))
                        print('Cleaning Cigarrillos...')
                        data = data.loc[:, data.columns.isin(keep)]
                        for x in data.columns:
                            if x in delete:
                                print(f'Dropping{x}')
                                data.drop(columns=x, inplace=True)
                            if x in rename.keys():
                                data.rename(columns={f'{x}':rename[f'{x}']},inplace=True)
                        data['Year']=f'FY{i}{i+1}'
                        if filename.endswith(f"Master_Picadura_FY{i}{i+1}(Septiembre).zip"):
                            print(f'Picadura FY{i}{i+1}')
                            path=os.path.join(dirpath, filename)
                            zipp=zipfile.ZipFile(path)
                            for y in zipp.namelist():
                                add=pd.read_excel(zipp.open(y))
                                add=add.loc[:, add.columns.isin(keepp)]
                                for x in data.columns:
                                    if x in rename.keys():
                                        add.rename(columns={f'{x}':rename[f'{x}']},inplace=True)
                            data=data.merge(add,left_on='customer_id',right_on='customer_id')
                        
                        lista.append(data)  
                        
    return  lista

In [5]:
dfs = [df.set_index('customer_id') for df in concat_files('/home/carmencuadrado/Ironhack/Final_Project/Altadis')]

FY1415
Cleaning Cigarrillos...
DroppingASM
DroppingPromotor_Responsable
Droppingsapcode
Droppinglogista_segment
DroppingArea
DroppingIpos
DroppingAlias_Tienda
DroppingTOTAL JPS
DroppingTOTAL HABANOS
DroppingTOTAL BN
DroppingTOTAL GAULOISES
FY1011
Cleaning Cigarrillos...
DroppingTOTAL JPS
DroppingTOTAL HABANOS
DroppingTOTAL BN
DroppingTOTAL GAULOISES
FY1213
Cleaning Cigarrillos...
DroppingASM
DroppingPromotor_Responsable
Droppingsapcode
Droppinglogista_segment
DroppingArea
DroppingIpos
DroppingAlias_Tienda
DroppingTOTAL JPS
DroppingTOTAL HABANOS
DroppingTOTAL BN
DroppingTOTAL GAULOISES
FY1112
Cleaning Cigarrillos...
DroppingASM
DroppingPromotor_Responsable
Droppingsapcode
Droppinglogista_segment
DroppingArea
DroppingIpos
DroppingAlias_Tienda
DroppingTOTAL JPS
DroppingTOTAL HABANOS
DroppingTOTAL BN
DroppingTOTAL GAULOISES
FY1617
Cleaning Cigarrillos...
DroppingASM
DroppingPromotor_Responsable
Droppingsapcode
Droppinglogista_segment
DroppingArea
DroppingIpos
DroppingAlias_Tienda
DroppingT

KeyError: 'customer_id'

In [6]:
data_=pd.concat(dfs, axis=0, sort=True)
data_.reset_index(inplace=True)

NameError: name 'dfs' is not defined

In [6]:
data_cigarrillos=data_cigarrillos[~data_cigarrillos.isnull()]

In [7]:
data_cigarrillos['customer_id']=data_cigarrillos['customer_id'].apply(lambda x: re.findall('[\d].*',str(x)))
data_cigarrillos['customer_id']=data_cigarrillos['customer_id'].apply(lambda x: re.sub('\[','',str(x)))
data_cigarrillos['customer_id']=data_cigarrillos['customer_id'].apply(lambda x: re.sub('\]','',str(x)))
data_cigarrillos['customer_id']=data_cigarrillos['customer_id'].apply(lambda x: x[1:-1])

In [8]:
data_cigarrillos['customer_id']=data_cigarrillos['customer_id'].apply(lambda x: int(float(x)))

In [9]:
data_cigarrillos.groupby('Year').agg('sum')

,customer_id,BASTOS FILTRE 25S,BONCALO 20S,BROOKLYN AZUL KS 20S,BROOKLYN ROJO KS 20S,DAVIDOFF CLASSIC,PostalCode,Ranking_Ncal,Ranking_Prov,Ranking_area,...,TOTAL FORTUNA,TOTAL ITG,TOTAL NEGROS ITG,TOTAL NOBEL,TOTAL RUBIOS ITG,TOTAL_MERCADO_CIGARRILLOS,TOTAL_MERCADO_NEGROS,TOTAL_MERCADO_RUBIOS,Total_WEST_BROOKLYN,Zip code
Year,,,,,,,,,,,,,,,,,,,,,
FY1011,336776556784,9608725.0,10901240.0,20959680.0,77582160.0,11336580.0,340891709.0,0.0,0.0,0.0,...,6287606390,21132743096,4688573795,3193379980,16444169301,62741533019,5433080119,57308452900,0.000000e+00,0.0
FY1112,327739662860,0.0,0.0,0.0,0.0,0.0,0.0,88206987.0,2878533.0,28156489.0,...,5669091305,18274096732,3843536850,3008494520,14430559882,54951547817,4452392950,50499154867,0.000000e+00,331944115.0
FY1213,326097845793,0.0,0.0,0.0,0.0,0.0,0.0,87232056.0,2856177.0,30129713.0,...,4583066460,15629417041,3147983925,2555760480,12481433116,48341540477,3689350065,44652190412,0.000000e+00,330260900.0
FY1314,324262360250,0.0,0.0,0.0,0.0,0.0,0.0,86336356.0,2839598.0,29771396.0,...,4301164996,14749931646,2810241200,2438710980,11939690446,46528144340,3301983068,43226161272,0.000000e+00,328407964.0
FY1415,321960512033,0.0,0.0,0.0,0.0,0.0,0.0,85131627.0,2808559.0,29330310.0,...,4120404919,14210726216,2561320840,2407425195,11649405376,46135552269,3001613300,43133938969,0.000000e+00,326072365.0
FY1516,320158450764,0.0,0.0,0.0,0.0,0.0,0.0,84295232.0,2794619.0,28188084.0,...,4001335073,13842153747,2378747635,2349727277,11463406112,46144152636,2780548121,43363604515,0.000000e+00,324219452.0
FY1617,317683170137,0.0,0.0,0.0,0.0,0.0,0.0,83027553.0,2765019.0,23491476.0,...,3932013106,13163250346,2128278530,2275987541,11034971816,44959524229,2488168833,42471355396,1.413078e+09,321704666.0
FY1718,315509290429,0.0,0.0,0.0,0.0,0.0,0.0,82028395.0,2741551.0,22671522.0,...,3859139558,12716377367,1927900855,2222434391,10788476512,44246877399,2250231684,41996645715,1.482205e+09,319506523.0
FY1819,312694637890,0.0,0.0,0.0,0.0,0.0,0.0,80739113.0,2711537.0,22304487.0,...,3823628078,12578105154,1774474840,2224302341,10803630314,44484417498,2069922354,42414495144,1.618823e+09,316648987.0


In [13]:
df = reduce(lambda df1,df2: pd.merge(df1,df2,on='id'), dfs)

KeyError: 'id'